# skips

In [11]:
import random
from random import sample, choice, randint
from avrasm import *

random.seed(42)

In [12]:
def setreg(reg, val):
    """read a known value into any register"""
    return LDS(r(reg), a(0)).read(val, 0)(f"set r{reg} to {val}")

def checkreg(reg, val):
    return STS(a(0), r(reg)).write(val, 0)(f"r{reg} should be {val}")

def setstatus(bit, isSet=True):
    if isSet:
        return BSET(bit)
    else:
        return BCLR(bit)

def checkstatus(bit, isSet=True, label="done"):
    mssg = f"bit {bit} of status should be {'set' if isSet else 'clear'}"
    if isSet:
        return BRBC(bit, label)(mssg)
    else:
        return BRBS(bit, label)(mssg)

In [13]:
def testCpse(regs, vals, instr):
    """test CPSE by setting two registers to values,
    and seeing if instruction i is skipped
    """
    reg1, reg2 = regs
    val1, val2 = vals
    skip = val1 == val2
    mssg = f"{val1} {'==' if skip else '!='} {val2}"
    if (skip):
        instr.skip()
    return [
        setreg(reg1, val1),
        setreg(reg2, val2),
        CPSE(r(reg1), r(reg2)),
        instr(mssg)
    ]

In [14]:
test_val = 0x97
cpseTests = [
    "\n;PREPROCESS TestCPSE",
    LDI(r(31), v(0x00)),
    LDI(r(30), v(0x00)),
    "; skip a LD (1 word) instruction",
    testCpse(sample(range(1, 30), 2), (0, 0), LD(r(0), 'Z').read(test_val, 0)),
    "; don't skip a LD instruction",
    testCpse(sample(range(1, 30), 2), (0, randint(0,0xFF)), LD(r(0), 'Z').read(test_val, 0)),
    "; skip a STS (2 word) instruction",
    testCpse(sample(range(1, 32), 2), (0, 0), checkreg(0,test_val)),
    "; don't skip a STS instruction",
    testCpse(sample(range(1, 32), 2), (randint(0,0xFE), 0xFF), checkreg(0,test_val)),
]

In [15]:
def testSbrc(reg, val, bit, instr):
    skip = not(val & (1<<bit))
    mssg = f"bit {bit} of {val:08b} is {'clear' if skip else 'set'}"
    if skip:
        instr.skip()
    return [
        setreg(reg, val),
        SBRC(r(reg), str(bit)),
        instr(mssg)
    ]

In [16]:
test_val = 0x73
sbrcTests = [
    "\n;PREPROCESS TestSBRC",
    LDI(r(31), v(0x00)),
    LDI(r(30), v(0x00)),
    "; skip a LD (1 word) instruction",
    testSbrc(choice(range(1, 30)), 0xF0, 0, LD(r(0), 'Z').read(test_val, 0)),
    "; don't skip a LD instruction",
    testSbrc(choice(range(1, 30)), 0x0F, 3, LD(r(0), 'Z').read(test_val, 0)),
    "; skip a STS (2 word) instruction",
    testSbrc(choice(range(1, 32)), 0xF0, 1, checkreg(0,test_val)),
    "; don't skip a STS instruction",
    testSbrc(choice(range(1, 32)), 0x0F, 2, checkreg(0,test_val)),
]

In [17]:
def testSbrs(reg, val, bit, instr):
    skip = (val & (1<<bit))
    mssg = f"bit {bit} of {val:08b} is {'set' if skip else 'clear'}"
    if skip:
        instr.skip()
    return [
        setreg(reg, val),
        SBRS(r(reg), str(bit)),
        instr(mssg)
    ]

In [18]:
test_val = 0x2A
sbrsTests = [
    "\n;PREPROCESS TestSBRS",
    LDI(r(31), v(0x00)),
    LDI(r(30), v(0x00)),
    "; don't skip a LD (1 word) instruction",
    testSbrs(choice(range(1, 30)), 0xF0, 4, LD(r(0), 'Z').read(test_val, 0)),
    "; skip a LD instruction",
    testSbrs(choice(range(1, 30)), 0x0F, 7, LD(r(0), 'Z').read(test_val, 0)),
    "; don't skip a STS (2 word) instruction",
    testSbrs(choice(range(1, 32)), 0xF0, 5, checkreg(0,test_val)),
    "; skip a STS instruction",
    testSbrs(choice(range(1, 32)), 0x0F, 6, checkreg(0,test_val)),
]

In [19]:
bits = [randint(0,1) == 0 for _ in range(8)]
vals = [randint(0, 0xFF) for _ in range(32)]
nopTests = [
    "\n;PREPROCESS TestNOP",
    "; set some random values in registers",
    [setstatus(i, bits[i]) for i in range(8)],
    [setreg(i, vals[i]) for i in range(32)],
    "; do a whole lot of nothing",
    NOP(),NOP(),NOP(),
    "; check that indeed nothing happened",
    [checkreg(i, vals[i]) for i in range(32)],
    [checkstatus(i, bits[i]) for i in range(8)],
    NOP()["done"]
]

In [20]:
doc = """; Tests the flow control skip instructions and NOP
THIS FILE IS GENERATED
for explanations or modifications, see notebook flowskip.ipynb
"""
skipTests = [
    doc,
    cpseTests,
    sbrcTests,
    sbrsTests,
    nopTests
]
with open("flow_skip.asm", "w") as f:
    f.write(stringify(skipTests))